In [1]:
%matplotlib inline

import os
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from numpy.linalg import eig, inv

from scipy import interpolate, spatial

import seaborn as sns

import cv2

In [2]:
""" ============== Golbal variables =============="""
global RcellColors
RcellColors = ['#00FFFF', '#1FF509', '#FF0000', '#CFCF1C', '#FF00FF', '#FFAE01']

In [3]:
"""============== Functions =============="""

'============== Functions =============='

In [4]:
"""Ellipse fitting"""

'Ellipse fitting'

In [5]:
### Ellipse fitting function from online
# source: http://nicky.vanforeest.com/misc/fitEllipse/fitEllipse.html
def fitEllipse(x,y):
    x = x[:,np.newaxis]
    y = y[:,np.newaxis]
    D =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
    S = np.dot(D.T,D)
    C = np.zeros([6,6])
    C[0,2] = C[2,0] = 2; C[1,1] = -1
    E, V =  eig(np.dot(inv(S), C))
    n = np.argmax(np.abs(E))
    a = V[:,n]
    return a


def ellipse_center(a):
    b,c,d,f,g,a = a[1]/2, a[2], a[3]/2, a[4]/2, a[5], a[0]
    num = b*b-a*c
    x0=(c*d-b*f)/num
    y0=(a*f-b*d)/num
    return np.array([x0,y0])


def ellipse_angle_of_rotation2( a ):
    b,c,d,f,g,a = a[1]/2, a[2], a[3]/2, a[4]/2, a[5], a[0]
    if b == 0:
        if a > c:
            return 0
        else:
            return np.pi/2
    else:
#         if a > c:
#             return np.arctan(2*b/(a-c))/2
#         else:
#             return np.pi/2 + np.arctan(2*b/(a-c))/2
        return np.arctan(2*b/(a-c))/2

def ellipse_axis_length( a ):
    b,c,d,f,g,a = a[1]/2, a[2], a[3]/2, a[4]/2, a[5], a[0]
    up = 2*(a*f*f+c*d*d+g*b*b-2*b*d*f-a*c*g)
    down1=(b*b-a*c)*( (c-a)*np.sqrt(1+4*b*b/((a-c)*(a-c)))-(c+a))
    down2=(b*b-a*c)*( (a-c)*np.sqrt(1+4*b*b/((a-c)*(a-c)))-(c+a))
    res1=np.sqrt(up/down1)
    res2=np.sqrt(up/down2)
    return np.array([res1, res2])

### my own to get summary
def fit_ellipse_custome(x,y):
    a = fitEllipse(x,y)
    center = ellipse_center(a)
    axis = ellipse_axis_length(a)
    phi = ellipse_angle_of_rotation2(a)
    return center, axis, phi

In [6]:
### custome ellipse fitting
def getInterpolation(points):
    points = points.astype(float)
    tck, u = interpolate.splprep([points[:,0], points[:,1]], s=0)
    unew = np.arange(0, 1.01, 0.0001)
    out = interpolate.splev(unew, tck)
    newPoints = np.vstack((out[0], out[1])).T
    return newPoints

def getEllipseXY(center, axis, phi):
    arc = 1
    R = np.arange(0,arc*2*np.pi, 0.01)
    a = axis[0]
    b = axis[1]
    xx = center[0] + a*np.cos(R)*np.cos(phi) - b*np.sin(R)*np.sin(phi)
    yy = center[1] + a*np.cos(R)*np.sin(phi) + b*np.sin(R)*np.cos(phi)
    ellipse = np.vstack((xx, yy)).T
    return ellipse

def fitBundle(points):
    # interpolate
    newPoints = getInterpolation(points)

    # fit ellipse using custome code
    center, axis, phi =  fit_ellipse_custome(newPoints[:,0], newPoints[:,1])

    # get ellipse points
    arc = 1
    R = np.arange(0,arc*2*np.pi, 0.01)
    ellipseXY = getEllipseXY(center, axis, phi)
    
    return center, axis, phi, ellipseXY

In [7]:
""" Inner angle calculation """
# source: https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def inner_angle(v1, v2, isRadians):
    """ Returns the angle in radians(or degree) between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    if isRadians:
        return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    else:
        return np.rad2deg(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

In [8]:
"""Dataframe processing"""

'Dataframe processing'

In [9]:
def group_headers(df, header_tag, isContain):
    '''What the function name says'''
    if isContain:
        return [col for col in df.columns.values if header_tag in col]
    else:
        return [col for col in df.columns.values if header_tag not in col]

In [10]:
def getNonNANlist(ls):
    return [i for i in ls if (not pd.isnull(i))]

In [11]:
def getROIdic(df_annot_control, dataPrefix, expName):
    region_list = getNonNANlist(list(df_annot_control.Region_Name.unique()))
    ROIdatas = {}
    for i in range(len(region_list)):
        region_index = i
        roi_path = os.path.join(dataPrefix, expName)
        ROI_name = region_list[i] + '.csv'
        df_tmp = pd.read_csv(os.path.join(roi_path, ROI_name))
        df_tmp.rename(columns = {' ':'No'}, inplace = True)
        ROIdatas[region_index] = df_tmp
    return ROIdatas

In [12]:
def concatinateROI(df_ROIdata, df_annot, ROIno):
    ### region lists
    region_list = getNonNANlist(list(df_annot.Region_Name.unique()))
    region_name = region_list[ROIno]
    
    ### get ROI data grouped by label
    df_ROIgroup = df_ROIdata.groupby('Label').mean()
    df_ROIgroup.sort_values('No', inplace=True)
    df_ROIgroup.reset_index(inplace=True)

    ### initialization
    coordList = []
    for i in range(6):
        for j in ['X', 'Y']:
            coordList.append('coor_R' + str(i+1) + '_' + j)
    RhellsCols = ['Bundle_No', 'num_Rcells'] + coordList
    extensionCols = ['ROI_No', 'Start_X', 'Start_Y', 'End_X', 'End_Y', 'Angle', 'Length']
    Rhells_df = pd.DataFrame(columns = RhellsCols);
    extension_df = pd.DataFrame(columns = extensionCols);
    
    ### update bundle and extension dataframe
    for ind in df_ROIgroup.index:
        if((df_ROIgroup.loc[ind,'Angle'] == 0) & (df_ROIgroup.loc[ind,'Length'] == 0)):
            df_tmp = pd.DataFrame(columns = RhellsCols)
            df_tmp.loc[0,'Bundle_No'] = int(ind+1)
            df_bd = df_ROIdata.loc[df_ROIdata['Label'] == list(df_ROIgroup.Label)[ind]]
            df_tmp.loc[0,'num_Rcells'] = int(df_bd.shape[0])
            if(len(df_bd) != 6):
                print('Not 6 R cells in bundle No. ' + str(ind+1))
#                 df_annot.loc[(df_annot['Bundle_No'] == ind+1) & (df_annot['Region_Name'] == region_name), 'Cof_Bundle'] = 0
            if(len(df_bd) <= 6):
                for i in range(len(df_bd)):
                    df_tmp.loc[0,['coor_R' + str(i+1) + '_X']] = float(df_bd.loc[df_bd.index[i], 'X'])
                    df_tmp.loc[0,['coor_R' + str(i+1) + '_Y']] = float(df_bd.loc[df_bd.index[i], 'Y'])
            else:
                for i in range(6):
                    df_tmp.loc[0,['coor_R' + str(i+1) + '_X']] = float(df_bd.loc[df_bd.index[i], 'X'])
                    df_tmp.loc[0,['coor_R' + str(i+1) + '_Y']] = float(df_bd.loc[df_bd.index[i], 'Y'])
            Rhells_df = Rhells_df.append(df_tmp, ignore_index=True)
        else:
            df_tmp = pd.DataFrame(columns = extensionCols);

            angle = np.radians(df_ROIgroup.loc[ind, 'Angle']) # angle in radias
            length = df_ROIgroup.loc[ind, 'Length']
            xm = df_ROIgroup.loc[ind, 'X']
            ym = df_ROIgroup.loc[ind, 'Y']

            df_tmp.loc[0,'ROI_No'] = int(ind+1)
            df_tmp.loc[0,'Angle'] = float(angle)
            df_tmp.loc[0,'Length'] = float(length)
#             df_tmp.loc[0,'Start_X'] = xm - length/2 * np.cos(angle)
#             df_tmp.loc[0,'Start_Y'] = ym - length/2 * np.sin(angle)
#             df_tmp.loc[0,'End_X'] = xm + length/2 * np.cos(angle)
#             df_tmp.loc[0,'End_Y'] = ym + length/2 * np.sin(angle)
            extension_df = extension_df.append(df_tmp, ignore_index=True)

    ### reset index
    Rhells_df.set_index('Bundle_No', inplace=True)
    extension_df.set_index('ROI_No', inplace=True)
    
    ### return
    return Rhells_df, extension_df, df_annot

In [13]:
### fit ellipse / get centroid as center
def updateRheelsInfo(Rhells_df):
    RcellCoorColls = group_headers(Rhells_df, 'coor_', True)
    for bundleNo in Rhells_df.index:
        # get position of R1-R6
        points = np.reshape(Rhells_df.loc[bundleNo,RcellCoorColls].values, (6,2))

        # fit ellipse
        center, axis, phi, ellipseXY = fitBundle(points)

        if(np.isnan(axis).any()):
            # alternative: centroid of 6 points.
            center = np.sum(points, axis = 0)/6
            Rhells_df.loc[bundleNo, 'shape_CenterX'] = center[0]
            Rhells_df.loc[bundleNo, 'shape_CenterY'] = center[1]
        else:
            Rhells_df.loc[bundleNo, 'shape_CenterX'] = center[0]
            Rhells_df.loc[bundleNo, 'shape_CenterY'] = center[1]
            Rhells_df.loc[bundleNo, 'shape_MajorAxis'] = axis[0]
            Rhells_df.loc[bundleNo, 'shape_MinorAxis'] = axis[1]
            Rhells_df.loc[bundleNo, 'shape_phi'] = phi
        
        # bundle rotation axis: middle of R3-4 vs. R1-6 (pointing outwards)
        bundle_axis = (points[2,:] + points[3,:])/2 - (points[0,:] + points[5,:])/2
        Rhells_df.loc[bundleNo, 'bundle_axis_X'] = bundle_axis[0]
        Rhells_df.loc[bundleNo, 'bundle_axis_Y'] = bundle_axis[1]
    return Rhells_df

In [14]:
### Update angles_new version_dependencies
def calculateCaseID(isR3onBottom, tcEx, comp):
    caseID = -1
    if(isR3onBottom):
        if(tcEx[1] <= comp):
            caseID = 1
        else:
            caseID = 2
    else:
        if(tcEx[1] <= comp):
            caseID = 2
        else:
            caseID = 1

def calculateRelativeAngles(Rtype, iEx, cRx, cTs, axisT0, aTs, isR3onBottom):
    ## UnravEling cTs and aTs
    cT3 = cTs[0]
    cT4 = cTs[1]
    cT7 = cTs[2]
    cT0_ = cTs[3]

    aT3RmT7 = aTs[0]
    aT3RmT4 = aTs[1]
    aT4RmT7 = aTs[2]
    aT3RmT0_ = aTs[3]
    aT0_RmT7 = aTs[4]


    ### information about extension trajectory
    aEx = extension_df.loc[iEx, 'Angle']
    lEx = extension_df.loc[iEx, 'Length']
    
    # trajectory vector
    scEx = cRx # originate point of the vector
    tcEx = np.array([scEx[0] + lEx * np.cos(aEx) , scEx[1] + lEx * np.sin(aEx)]) # end point of the vector
    
    # project Rx trajectory vector to the middle-point
    scExp = cRm # originate point of the vector
    tcExp = np.array([scExp[0] + lEx * np.cos(aEx) , scExp[1] + lEx * np.sin(aEx)]) # end point of the vector

    ### inner angles
    # starting from middle point
    aExRmT7 = inner_angle(cT7-cRm, tcExp-scExp, True) # trajectory to T7
    aExRmT4 = inner_angle(cT4-cRm, tcExp-scExp, True) # trajectory to T4
    aExRmT3 = inner_angle(cT3-cRm, tcExp-scExp, True) # trajectory to T3
    aExRmT0_ = inner_angle(axisT0, tcExp-scExp, True) # extension to bundle axis
    
    # starting from Rx/Rx
    aExRxT7 = inner_angle(cT7-cRx, tcEx-scEx, True) # trajectory to T7
    aExRxT4 = inner_angle(cT4-cRx, tcEx-scEx, True) # trajectory to T4
    aExRxT3 = inner_angle(cT3-cRx, tcEx-scEx, True) # trajectory to T3
    
    aT3RxT4 = inner_angle(cT4 - cRx, cT3 - cRx, True) # angle from T3 to T4
    aT3RxT0_ = inner_angle(cT0_ - cRx, cT3 - cRx, True) # angle from T3 to T0_
    aT4RxT7 = inner_angle(cT7 - cRx, cT4 - cRx, True) # angle from T4 to T7

    ### values for determining conditions
    comp1 = getInterpolate(tcEx, cRx, cT4)
    comp2 = getInterpolate(tcEx, cRx, cT0_)
    comp3 = getInterpolate(tcExp, cRm, cT4)
    comp4 = getInterpolate(tcExp, cRm, cT0_)

    ### calculating relative angles
    # ver1: angle angle relative to T4, originate at Rx
    caseID_1 = calculateCaseID(isR3onBottom, tcEx, comp1)
    if(caseID_1 == 1):                        # entension between T3 and T4
        if(Rtype == 3):
            angleV1 = aExRxT4/aT3RxT4
        elif(Rtype == 4):
            angleV1 = - aExRxT4/aT3RxT4
    if(caseID_1 == 2):                        # entension between T4 and T7
        if(Rtype == 3):
            angleV1 = - aExRxT4/aT4RxT7
        elif(Rtype == 4):
            angleV1 = aExRxT4/aT4RxT7

    # ver2: angle angle relative to T0_, originate at Rx
    caseID_1 = calculateCaseID(isR3onBottom, tcEx, comp2)
    if(caseID_1 == 1):                        # entension between T3 and T4
        if(Rtype == 3):
            angleV2 = aExRxT0_/aT3RxT0_
        elif(Rtype == 4):
            angleV2 = - aExRxT0_/aT3RxT0_
    if(caseID_1 == 2):                        # entension between T4 and T7
        if(Rtype == 3):
            angleV2 = - aExRxT0_/aT0_RxT7
        elif(Rtype == 4):
            angleV2 = aExRxT0_/aT0_RxT7

    # ver3: angle relative to T4, originate at Rm
    caseID_1 = calculateCaseID(isR3onBottom, tcEx, comp3)
    if(caseID_1 == 1):                        # entension between T3 and T4
        if(Rtype == 3):
            angleV3 = aExRmT4/aT3RmT4
        elif(Rtype == 4):
            angleV3 = - aExRmT4/aT3RmT4
    if(caseID_1 == 2):                        # entension between T4 and T7
        if(Rtype == 3):
            angleV3 = - aExRmT4/aT4RmT7
        elif(Rtype == 4):
            angleV3 = aExRmT4/aT4RmT7

    # ver4: angle relative to T0_, originate at Rm
    caseID_4 = calculateCaseID(isR3onBottom, tcEx, comp4)
    if(caseID_4 == 1):                        # entension between T3 and T4
        if(Rtype == 3):
            angleV4 = aExRmT0_/aT3RmT0_
        elif(Rtype == 4):
            angleV4 = - aExRmT0_/aT3RmT0_
    if(caseID_4 == 2):                        # entension between T4 and T7
        if(Rtype == 3):
            angleV4 = - aExRmT0_/aT0_RmT7
        elif(Rtype == 4):
            angleV4 = aExRmT0_/aT0_RmT7

    ### calculate putative target
    if(min(aExRxT3, aExRxT4, aExRxT7) == aExRxT3):
        pT_Rx = 'T3'
    elif(min(aExRxT3, aExRxT4, aExRxT7) == aExRxT4):
        pT_Rx = 'T4'
    else:
        pT_Rx = 'T3_'

    relAngles = [angleV1, angleV2, angleV3, angleV4]
    angles = [aExRxT3, aExRxT4, aExRxT7, aExRmT0_]

    return aEx, lEx, pT_Rx, scEx, tcEx, angles, relAngles

In [15]:
### Update angles_new version
def updateAnglesInfo(df_annot, Rheels_df, extension_df, region_name, isPrint):
    ### create bundles_df dataframe
    newCols = ['Extension_GFP_R3pre',
       'Extension_GFP_R4pre', 'Extension_target_R3pre',
       'Extension_target_R4pre', 'Extension_RFP_R3pre', 'Extension_RFP_R4pre',
       'Extension_angle_R3pre', 'Extension_angle_R3pre_toBundle',
       'Extension_angle_R3pre_toTarget', 'Extension_angle_R3pre_ver3', 'Extension_angle_R4pre',
       'Extension_angle_R4pre_toBundle', 'Extension_angle_R4pre_toTarget', 'Extension_angle_R4pre_ver3',
       'Extension_length_R3pre', 'Extension_length_R4pre',
       'Extension_relaLength_R3pre', 'Extension_relaLength_R4pre',
       'bundle_axis_X', 'bundle_axis_Y', 'bundle_center_X', 'bundle_center_Y']
    cof_cols = group_headers(df_annot, 'Cof_', True)
    newCols = newCols + cof_cols
    bundles_df = pd.DataFrame(columns = newCols)

    ### GFP & RFP: from string to numbers
    GR_mapping = {'W':1, 'M':2, 'S':3}

    ### loop through annotations
    for ind in df_annot[df_annot['Region_Name'] == region_name].index:
        
        if(isPrint):
            print(ind)
        
        ### if the bundle have R3/4 extensions
        if((not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])) | (not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])) ):
            ### create new dataframe space
            df_tmp = pd.DataFrame(columns = newCols)

            ### originating bundle
            # bundle number
            bundle_No = df_annot.iloc[ind]['Bundle_No']
            if(isPrint):
                print('Bundle No: ' + str(bundle_No))
            # center of originating bundle
            cT0 = np.array([ Rheels_df.loc[bundle_No,'shape_CenterX'], Rheels_df.loc[bundle_No,'shape_CenterY'] ])

            # bundle axis
            axisT0 = np.array([ Rheels_df.loc[bundle_No, 'bundle_axis_X'], Rheels_df.loc[bundle_No, 'bundle_axis_Y'] ])

            # position of R3 & R4
            cR3 = np.array([ Rheels_df.loc[bundle_No, 'coor_R3_X'], Rheels_df.loc[bundle_No, 'coor_R3_Y'] ])
            cR4 = np.array([ Rheels_df.loc[bundle_No, 'coor_R4_X'], Rheels_df.loc[bundle_No, 'coor_R4_Y'] ])

            # middle point between R3 and R4
            cRm = (cR3 + cR4)/2
            cT0_ = cRm + axisT0
            
            # relative position of R3 and R4
            if(cR3[1] <= cR4[1]):
                isR3onBottom = True
            else:
                isR3onBottom = False

            # update bundles_df:
            df_tmp.loc[0, 'bundleNo'] = int(bundle_No)
            df_tmp.loc[0, 'bundle_center_X'] = float(cT0[0])
            df_tmp.loc[0, 'bundle_center_Y'] = float(cT0[1])
            df_tmp.loc[0, 'bundle_axis_X'] = float(axisT0[0])
            df_tmp.loc[0, 'bundle_axis_Y'] = float(axisT0[1])

            ### putative targets
            # putative target index
            iT3 = int(df_annot.iloc[ind]['Target_T3'])
            iT4 = int(df_annot.iloc[ind]['Target_T4'])
            iT7 = int(df_annot.iloc[ind]['Target_T7'])

            # target position
            cT3 = np.array([ Rheels_df.loc[iT3,'shape_CenterX'], Rheels_df.loc[iT3,'shape_CenterY'] ])
            cT4 = np.array([ Rheels_df.loc[iT4,'shape_CenterX'], Rheels_df.loc[iT4,'shape_CenterY'] ])
            cT7 = np.array([ Rheels_df.loc[iT7,'shape_CenterX'], Rheels_df.loc[iT7,'shape_CenterY'] ])
            cTs = [cT3, cT4, cT7, cT0_]

            # angle between three targets
            aT3RmT7 = inner_angle(cT3 - cRm, cT7 - cRm, True)
            
            aT3RmT4 = inner_angle(cT4 - cRm, cT3 - cRm, True) # angle from T3 to T4
            aT4RmT7 = inner_angle(cT7 - cRm, cT4 - cRm, True) # angle from T4 to T7
            
            aT3RmT0_ = inner_angle(cT0_ - cRm, cT3 - cRm, True) # angle from T3 to T0'
            aT0_RmT7 = inner_angle(cT7 - cRm, cT0_ - cRm, True) # angle from T0' to T7
            
            aTs = [aT3RmT7, aT3RmT4, aT4RmT7, aT3RmT0_, aT0_RmT7]

            ### extensions: R3precursor
            if(not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])):
                # info about R3pre extension
                iE3 = int(df_annot.iloc[ind]['Extension_R3pre'])
                if(isPrint):
                    print('R3: ' + str(iE3))

                # angles calculation
                aEx, lEx, pT_Rx, scEx, tcEx, angles, relAngles = calculateRelativeAngles(3, iE3, cR3, cTs, axisT0, aTs, isR3onBottom)
                
                # update extension_df
                extension_df.loc[iE3, 'Start_X'] = float(scEx[0])
                extension_df.loc[iE3, 'Start_Y'] = float(scEx[1])
                extension_df.loc[iE3, 'End_X'] = float(tcEx[0])
                extension_df.loc[iE3, 'End_Y'] = float(tcEx[1])
                extension_df.loc[iE3, 'innerAngle_T3'] = float(angles[0])
                extension_df.loc[iE3, 'innerAngle_T4'] = float(angles[1])
                extension_df.loc[iE3, 'innerAngle_T7'] = float(angles[2])
                extension_df.loc[iE3, 'innerAngle_bundleAxis'] = float(angles[3])
    
                # update putative target
                df_annot.loc[ind, 'Extension_target_R3pre'] = pT_Rx
                df_tmp.loc[0, 'Extension_target_R3pre'] = pT_Rx

                # update bundles_df:
                df_tmp.loc[0, 'Extension_angle_R3pre'] = float(aEx)
                df_tmp.loc[0, 'Extension_length_R3pre'] = float(lEx)
                
                df_tmp.loc[0, 'Extension_angle_relative_R3pre_ver1'] = relAngles[0]
                df_tmp.loc[0, 'Extension_angle_relative_R3pre_ver2'] = relAngles[1]
                df_tmp.loc[0, 'Extension_angle_relative_R3pre_ver3'] = relAngles[2]
                df_tmp.loc[0, 'Extension_angle_relative_R3pre_ver4'] = relAngles[3]
                
                if(not pd.isnull(df_annot.iloc[ind]['GFP_R3pre'])):
                    df_tmp.loc[0, 'Extension_GFP_R3pre'] = GR_mapping[df_annot.iloc[ind]['GFP_R3pre']]
                if(not pd.isnull(df_annot.iloc[ind]['RFP_R3pre'])):
                    df_tmp.loc[0, 'Extension_RFP_R3pre'] = GR_mapping[df_annot.iloc[ind]['RFP_R3pre']]

        ### extensions: R4precursor
        if(not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])):
            # info about R4pre extension
                iE4 = int(df_annot.iloc[ind]['Extension_R4pre'])
                if(isPrint):
                    print('R4: ' + str(iE3))

                # angles calculation
                aEx, lEx, pT_Rx, scEx, tcEx, angles, relAngles = calculateRelativeAngles(4, iE4, cR4, cTs, axisT0, aTs, isR3onBottom)
                
                # update extension_df
                extension_df.loc[iE4, 'Start_X'] = float(scEx[0])
                extension_df.loc[iE4, 'Start_Y'] = float(scEx[1])
                extension_df.loc[iE4, 'End_X'] = float(tcEx[0])
                extension_df.loc[iE4, 'End_Y'] = float(tcEx[1])
                extension_df.loc[iE4, 'innerAngle_T3'] = float(angles[0])
                extension_df.loc[iE4, 'innerAngle_T4'] = float(angles[1])
                extension_df.loc[iE4, 'innerAngle_T7'] = float(angles[2])
                extension_df.loc[iE4, 'innerAngle_bundleAxis'] = float(angles[3])
    
                # update putative target
                df_annot.loc[ind, 'Extension_target_R4pre'] = pT_Rx
                df_tmp.loc[0, 'Extension_target_R4pre'] = pT_Rx

                # update bundles_df:
                df_tmp.loc[0, 'Extension_angle_R4pre'] = float(aEx)
                df_tmp.loc[0, 'Extension_length_R4pre'] = float(lEx)
                
                df_tmp.loc[0, 'Extension_angle_relative_R4pre_ver1'] = relAngles[0]
                df_tmp.loc[0, 'Extension_angle_relative_R4pre_ver2'] = relAngles[1]
                df_tmp.loc[0, 'Extension_angle_relative_R4pre_ver3'] = relAngles[2]
                df_tmp.loc[0, 'Extension_angle_relative_R4pre_ver4'] = relAngles[3]
                
                if(not pd.isnull(df_annot.iloc[ind]['GFP_R4pre'])):
                    df_tmp.loc[0, 'Extension_GFP_R4pre'] = GR_mapping[df_annot.iloc[ind]['GFP_R4pre']]
                if(not pd.isnull(df_annot.iloc[ind]['RFP_R4pre'])):
                    df_tmp.loc[0, 'Extension_RFP_R4pre'] = GR_mapping[df_annot.iloc[ind]['RFP_R4pre']]

        if((not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])) & (not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])) ) :
            # whether extension is symmetrical:
            #df_annot.loc[ind, 'Extension_symmetricity'] = aE3T0 - aE4T0
            # relative length
            df_tmp.loc[0, 'Extension_relaLength_R3pre'] = lE3/lE4
            df_tmp.loc[0, 'Extension_relaLength_R4pre'] = 1

        ### confidences.
        df_tmp.loc[0, cof_cols] = df_annot.iloc[ind][cof_cols].values.astype('int')
        
        ### whether off-grid or not.
        df_tmp.loc[0, 'Off_grid'] = df_annot.iloc[ind]['Off_grid'].astype('int')
        
        ### append temp to bundle_df
        bundles_df = bundles_df.append(df_tmp, ignore_index=True)

    ### further clean up bundles_df
    # set bundleNo as index
    bundles_df.set_index('bundleNo', inplace=True) 

    # remove 'NaN' strings
    bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_GFP_R3pre']) == True,'Extension_GFP_R3pre'] = 0
    bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_RFP_R3pre']) == True,'Extension_RFP_R3pre'] = 0
    bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_GFP_R4pre']) == True,'Extension_GFP_R4pre'] = 0
    bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_RFP_R4pre']) == True,'Extension_RFP_R4pre'] = 0
    
    return df_annot, Rheels_df, extension_df, bundles_df

In [16]:
### Update angles_old version
def updateAnglesInfo(df_annot, Rheels_df, extension_df, region_name, isPrint):
    ### create bundles_df dataframe
    newCols = ['Extension_GFP_R3pre',
       'Extension_GFP_R4pre', 'Extension_target_R3pre',
       'Extension_target_R4pre', 'Extension_RFP_R3pre', 'Extension_RFP_R4pre',
       'Extension_angle_R3pre', 'Extension_angle_R3pre_toBundle',
       'Extension_angle_R3pre_toTarget', 'Extension_angle_R3pre_ver3', 'Extension_angle_R4pre',
       'Extension_angle_R4pre_toBundle', 'Extension_angle_R4pre_toTarget', 'Extension_angle_R4pre_ver3',
       'Extension_length_R3pre', 'Extension_length_R4pre',
       'Extension_relaLength_R3pre', 'Extension_relaLength_R4pre',
       'bundle_axis_X', 'bundle_axis_Y', 'bundle_center_X', 'bundle_center_Y']
    cof_cols = group_headers(df_annot, 'Cof_', True)
    newCols = newCols + cof_cols
    bundles_df = pd.DataFrame(columns = newCols)

    ### GFP & RFP: from string to numbers
    GR_mapping = {'W':1, 'M':2, 'S':3}

    ### loop through annotations
    for ind in df_annot[df_annot['Region_Name'] == region_name].index:
        
        if(isPrint):
            print(ind)
        
        ### if the bundle have R3/4 extensions
        if((not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])) | (not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])) ):
            ### create new dataframe space
            df_tmp = pd.DataFrame(columns = newCols)

            ### originating bundle
            # bundle number
            bundle_No = df_annot.iloc[ind]['Bundle_No']
            if(isPrint):
                print('Bundle No: ' + str(bundle_No))
            # center of originating bundle
            cT0 = np.array([ Rheels_df.loc[bundle_No,'shape_CenterX'], Rheels_df.loc[bundle_No,'shape_CenterY'] ])

            # bundle axis
            axisT0 = np.array([ Rheels_df.loc[bundle_No, 'bundle_axis_X'], Rheels_df.loc[bundle_No, 'bundle_axis_Y'] ])

            # position of R3 & R4
            cR3 = np.array([ Rheels_df.loc[bundle_No, 'coor_R3_X'], Rheels_df.loc[bundle_No, 'coor_R3_Y'] ])
            cR4 = np.array([ Rheels_df.loc[bundle_No, 'coor_R4_X'], Rheels_df.loc[bundle_No, 'coor_R4_Y'] ])

            # middle point between R3 and R4
            cRm = (cR3 + cR4)/2
            cT0_ = cRm + axisT0

            # update bundles_df:
            df_tmp.loc[0, 'bundleNo'] = int(bundle_No)
            df_tmp.loc[0, 'bundle_center_X'] = float(cT0[0])
            df_tmp.loc[0, 'bundle_center_Y'] = float(cT0[1])
            df_tmp.loc[0, 'bundle_axis_X'] = float(axisT0[0])
            df_tmp.loc[0, 'bundle_axis_Y'] = float(axisT0[1])

            ### putative targets
            # putative target index
            iT3 = int(df_annot.iloc[ind]['Target_T3'])
            iT4 = int(df_annot.iloc[ind]['Target_T4'])
            iT7 = int(df_annot.iloc[ind]['Target_T7'])

            # target position
            cT3 = np.array([ Rheels_df.loc[iT3,'shape_CenterX'], Rheels_df.loc[iT3,'shape_CenterY'] ])
            cT4 = np.array([ Rheels_df.loc[iT4,'shape_CenterX'], Rheels_df.loc[iT4,'shape_CenterY'] ])
            cT7 = np.array([ Rheels_df.loc[iT7,'shape_CenterX'], Rheels_df.loc[iT7,'shape_CenterY'] ])
            cTs = [cT3, cT4, cT7, cT0_]

            # angle between three targets
            aT3RmT7 = inner_angle(cT3 - cRm, cT7 - cRm, True)
            
            aT3RmT4 = inner_angle(cT4 - cRm, cT3 - cRm, True) # angle from T3 to T4
            aT4RmT7 = inner_angle(cT7 - cRm, cT4 - cRm, True) # angle from T4 to T7
            
            aT3RmT0_ = inner_angle(cT0_ - cRm, cT3 - cRm, True) # angle from T3 to T0'
            aT0_RmT7 = inner_angle(cT7 - cRm, cT0_ - cRm, True) # angle from T0' to T7
            
            aTs = [aT3RmT7, aT3RmT4, aT4RmT7, aT3RmT0_, aT0_RmT7]

        ### extensions: R3precursor
        if(not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])):
            if(isPrint):
                print('R3')
            # info about R3pre extension
            iE3 = int(df_annot.iloc[ind]['Extension_R3pre'])
            if(isPrint):
                print('R3: ' + str(iE3))
            aE3 = extension_df.loc[iE3, 'Angle']
            lE3 = extension_df.loc[iE3, 'Length']
            
            # extension vector
            scE3 = cR3 # originate point of the vector
            tcE3 = np.array([scE3[0] + lE3 * np.cos(aE3) , scE3[1] + lE3 * np.sin(aE3)]) # end point of the vector
            
            # project R3 extension vector to the middle-point
            scE3p = cRm # originate point of the vector
            tcE3p = np.array([scE3p[0] + lE3 * np.cos(aE3) , scE3p[1] + lE3 * np.sin(aE3)]) # end point of the vector

            # inner angles
            aE3RmT7 = inner_angle(cT7-cRm, tcE3p-scE3p, True) # extension to T7
            aE3RmT4 = inner_angle(cT4-cRm, tcE3p-scE3p, True) # extension to T4
            aE3RmT3 = inner_angle(cT3-cRm, tcE3p-scE3p, True) # extension to T3
            
            aE3R3T7 = inner_angle(cT7-cR3, tcE3-scE3, True) # extension to T7
            aE3R3T4 = inner_angle(cT4-cR3, tcE3-scE3, True) # extension to T4
            aE3R3T3 = inner_angle(cT3-cR3, tcE3-scE3, True) # extension to T3
            
            aE3RmT0_ = inner_angle(axisT0, tcE3p-scE3p, True) # extension to bundle axis
            aE3RmT4 = inner_angle(cT4 - cRm, tcE3p-scE3p, True)
            
            aT3R3T4 = inner_angle(cT4 - cR3, cT3 - cR3, True) # angle from T3 to T4
            aT4R3T7 = inner_angle(cT7 - cR3, cT4 - cR3, True) # angle from T4 to T7

            # angle relative to T4, originate at R3
            comp3 = getInterpolate(tcE3, cR3, cT4)
            if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 is at top.
                if(tcE3[1] <= comp3):               # entension between T3 and T4
                    angle1v1_R3 = aE3R3T4/aT3R3T4
                else:                               # entension between T4 and T7
                    angle1v1_R3 = - aE3R3T4/aT4R3T7
            else:                                   # R3 at top and R4 at bottom
                if(tcE3[1] >= comp3):               # entension between T3 and T4
                    angle1v1_R3 = aE3R3T4/aT3R3T4
                else:                               # entension between T4 and T7
                    angle1v1_R3 = - aE3R3T4/aT4R3T7
            df_tmp.loc[0, 'Extension_angle_R3pre_toTarget'] = float(angle1v1_R3)

            # angle relative to bundle axis, originate at Rm
            compp3 = getInterpolate(tcE3p, cRm, cT0_)
            if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 is at top.
                if(tcE3p[1] <= compp3):               # entension between T3 and T4
                    angle1v2_R3 = aE3RmT0_/aT3RmT0_
                else:                               # entension between T4 and T7
                    angle1v2_R3 = aE3RmT0_/aT0_RmT7
            else:                                   # R3 at top and R4 at bottom
                if(tcE3p[1] >= compp3):               # entension between T3 and T4
                    angle1v2_R3 = aE3RmT0_/aT3RmT0_
                else:                               # entension between T4 and T7
                    angle1v2_R3 = aE3RmT0_/aT0_RmT7
            df_tmp.loc[0, 'Extension_angle_R3pre_toBundle'] = float(angle1v2_R3)
            
            # angle ver3.
            aE3RmT4
            comppp3 = getInterpolate(tcE3p, cRm, cT4)
            if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 is at top.
                if(tcE3p[1] <= comppp3):               # entension between T3 and T4
                    angle1v3_R3 = aE3RmT4/aT3RmT4
                else:                               # entension between T4 and T7
                    angle1v3_R3 = aE3RmT4/aT4RmT7
            else:                                   # R3 at top and R4 at bottom
                if(tcE3p[1] >= comppp3):               # entension between T3 and T4
                    angle1v3_R3 = aE3RmT4/aT3RmT4
                else:                               # entension between T4 and T7
                    angle1v3_R3 = aE3RmT4/aT4RmT7
            df_tmp.loc[0, 'Extension_angle_R3pre_ver3'] = float(angle1v3_R3)
            
            # update extension_df:
            extension_df.loc[iE3, 'Start_X'] = float(scE3[0])
            extension_df.loc[iE3, 'Start_Y'] = float(scE3[1])
            extension_df.loc[iE3, 'End_X'] = float(tcE3[0])
            extension_df.loc[iE3, 'End_Y'] = float(tcE3[1])
            extension_df.loc[iE3, 'innerAngle_T3'] = float(aE3R3T3)
            extension_df.loc[iE3, 'innerAngle_T4'] = float(aE3R3T4)
            extension_df.loc[iE3, 'innerAngle_T7'] = float(aE3R3T7)
            extension_df.loc[iE3, 'innerAngle_bundleAxis'] = float(aE3RmT0_)

            # update putative target
            if(min(aE3R3T3, aE3R3T4, aE3R3T7) == aE3R3T3):
                pT_R3 = 'T3'
            elif(min(aE3R3T3, aE3R3T4, aE3R3T7) == aE3R3T4):
                pT_R3 = 'T4'
            else:
                pT_R3 = 'T3_'
            df_annot.loc[ind, 'Extension_target_R3pre'] = pT_R3
            df_tmp.loc[0, 'Extension_target_R3pre'] = pT_R3

            # update bundles_df:
            df_tmp.loc[0, 'Extension_angle_R3pre'] = float(aE3)
            df_tmp.loc[0, 'Extension_length_R3pre'] = float(lE3)
            if(not pd.isnull(df_annot.iloc[ind]['GFP_R3pre'])):
                df_tmp.loc[0, 'Extension_GFP_R3pre'] = GR_mapping[df_annot.iloc[ind]['GFP_R3pre']]
            if(not pd.isnull(df_annot.iloc[ind]['RFP_R3pre'])):
                df_tmp.loc[0, 'Extension_RFP_R3pre'] = GR_mapping[df_annot.iloc[ind]['RFP_R3pre']]

        ### extensions: R4precursor
        if(not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])):
            
            if(isPrint):
                print('R4')
                
            # info about R4pre extension
            iE4 = int(df_annot.iloc[ind]['Extension_R4pre'])
            if(isPrint):
                print('R4: ' + str(iE4))
            aE4 = extension_df.loc[iE4, 'Angle']
            lE4 = extension_df.loc[iE4, 'Length']
            
            # extension vector
            scE4 = cR4 # originate point of the vector
            tcE4 = np.array([scE4[0] + lE4 * np.cos(aE4) , scE4[1] + lE4 * np.sin(aE4)]) # end point of the vector
            
            # project R4 extension to the middle-point
            scE4p = cRm
            tcE4p = np.array([scE4p[0] + lE4 * np.cos(aE4) , scE4p[1] + lE4 * np.sin(aE4)])
            
            # inner angles
            aE4RmT7 = inner_angle(cT7-cRm, tcE4p-scE4p, True) # extension to T7
            aE4RmT4 = inner_angle(cT4-cRm, tcE4p-scE4p, True) # extension to T4
            aE4RmT3 = inner_angle(cT3-cRm, tcE4p-scE4p, True) # extension to T3
            
            aE4R4T7 = inner_angle(cT7-cR4, tcE4-scE4, True) # extension to T7
            aE4R4T4 = inner_angle(cT4-cR4, tcE4-scE4, True) # extension to T4
            aE4R4T3 = inner_angle(cT3-cR4, tcE4-scE4, True) # extension to T3
            
            aE4RmT0_ = inner_angle(axisT0, tcE4p-scE4p, True) # extension to bundle axis
            aE4RmT4 = inner_angle(cT4 - cRm, tcE4p-scE4p, True)
            
            aT3R4T4 = inner_angle(cT4 - cR4, cT3 - cR4, True) # angle from T3 to T4
            aT4R4T7 = inner_angle(cT7 - cR4, cT4 - cR4, True) # angle from T4 to T7
            
            # angle relative to T4, originate at R4
            comp4 = getInterpolate(tcE4, cR4, cT4)
            if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 at top
                if(tcE4[1] >= comp4):               # entension between T4 and T7
                    angle1v1_R4 = aE4R4T4/aT4R4T7
                else:                               # entension between T3 and T4
                    angle1v1_R4 = - aE4R4T4/aT3R4T4
            else:                                   # R3 at top and R4 at bottom
                if(tcE4[1] <= comp4):               # entension between T4 and T7
                    angle1v1_R4 = aE4R4T4/aT4R4T7
                else:                               # entension between T3 and T4
                    angle1v1_R4 = - aE4R4T4/aT3R4T4
            df_tmp.loc[0, 'Extension_angle_R4pre_toTarget'] = float(angle1v1_R4)
            
            # angle relative to bundle axis
            compp4 = getInterpolate(tcE4p, cRm, cT0_)
            if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 is at top.
                if(tcE4p[1] >= compp4):              # entension between T4 and T7
                    angle1v2_R4 = aE4RmT0_/aT0_RmT7
                else:                               # entension between T3 and T4
                    angle1v2_R4 = aE4RmT0_/aT3RmT0_
            else:                                   # R3 at top and R4 at bottom
                if(tcE4p[1] <= compp4):              # entension between T4 and T7
                    angle1v2_R4 = aE4RmT0_/aT0_RmT7
                else:                               # entension between T3 and T4
                    angle1v2_R4 = aE4RmT0_/aT3RmT0_
            df_tmp.loc[0, 'Extension_angle_R4pre_toBundle'] = float(angle1v2_R4)
            
            # angle ver3
            comppp4 = getInterpolate(tcE4p, cRm, cT4)
            if(cR3[1] <= cR4[1]):                   # R3 at bottom and R4 is at top.
                if(tcE4p[1] >= comppp4):              # entension between T4 and T7
                    angle1v3_R4 = aE4RmT4/aT4RmT7
                else:                               # entension between T3 and T4
                    angle1v3_R4 = aE4RmT4/aT3RmT4
            else:                                   # R3 at top and R4 at bottom
                if(tcE4p[1] <= comppp4):              # entension between T4 and T7
                    angle1v3_R4 = aE4RmT4/aT0_RmT7
                else:                               # entension between T3 and T4
                    angle1v3_R4 = aE4RmT4/aT3RmT4
            df_tmp.loc[0, 'Extension_angle_R4pre_ver3'] = float(angle1v3_R4)
            
            
            # update extension_df:
            extension_df.loc[iE4, 'Start_X'] = float(scE4[0])
            extension_df.loc[iE4, 'Start_Y'] = float(scE4[1])
            extension_df.loc[iE4, 'End_X'] = float(tcE4[0])
            extension_df.loc[iE4, 'End_Y'] = float(tcE4[1])
            extension_df.loc[iE4, 'innerAngle_T3'] = float(aE4R4T3)
            extension_df.loc[iE4, 'innerAngle_T4'] = float(aE4R4T4)
            extension_df.loc[iE4, 'innerAngle_T7'] = float(aE4R4T7)
            extension_df.loc[iE4, 'innerAngle_bundleAxis'] = float(aE4RmT0_)

            # update df_annot:
            if(min(aE4R4T3, aE4R4T4, aE4R4T7) == aE4R4T3):
                pT_R4 = 'T3'
            elif(min(aE4R4T3, aE4R4T4, aE4R4T7) == aE4R4T4):
                pT_R4 = 'T4'
            else:
                pT_R4 = 'T3_'
            df_annot.loc[ind, 'Extension_target_R4pre'] = pT_R4
            df_tmp.loc[0, 'Extension_target_R4pre'] = pT_R4

            # update bundles_df:
            df_tmp.loc[0, 'Extension_angle_R4pre'] = float(aE4)
            df_tmp.loc[0, 'Extension_length_R4pre'] = float(lE4)
            if(not pd.isnull(df_annot.iloc[ind]['GFP_R4pre'])):
                df_tmp.loc[0, 'Extension_GFP_R4pre'] = GR_mapping[df_annot.iloc[ind]['GFP_R4pre']]
            if(not pd.isnull(df_annot.iloc[ind]['RFP_R4pre'])):
                df_tmp.loc[0, 'Extension_RFP_R4pre'] = GR_mapping[df_annot.iloc[ind]['RFP_R4pre']]
            #df_tmp.loc[0, 'Extension_angle_R4pre_toBundle'] = aE4T0/aT3T0T7

        if((not pd.isnull(df_annot.iloc[ind]['Extension_R3pre'])) & (not pd.isnull(df_annot.iloc[ind]['Extension_R4pre'])) ) :
            # whether extension is symmetrical:
            #df_annot.loc[ind, 'Extension_symmetricity'] = aE3T0 - aE4T0
            # relative length
            df_tmp.loc[0, 'Extension_relaLength_R3pre'] = lE3/lE4
            df_tmp.loc[0, 'Extension_relaLength_R4pre'] = 1

        ### confidences.
        df_tmp.loc[0, cof_cols] = df_annot.iloc[ind][cof_cols].values.astype('int')
        
        ### whether off-grid or not.
        df_tmp.loc[0, 'Off_grid'] = df_annot.iloc[ind]['Off_grid'].astype('int')
        
        ### append temp to bundle_df
        bundles_df = bundles_df.append(df_tmp, ignore_index=True)

    ### further clean up bundles_df
    # set bundleNo as index
    bundles_df.set_index('bundleNo', inplace=True) 

    # remove 'NaN' strings
    bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_GFP_R3pre']) == True,'Extension_GFP_R3pre'] = 0
    bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_RFP_R3pre']) == True,'Extension_RFP_R3pre'] = 0
    bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_GFP_R4pre']) == True,'Extension_GFP_R4pre'] = 0
    bundles_df.loc[pd.isnull(bundles_df.loc[:,'Extension_RFP_R4pre']) == True,'Extension_RFP_R4pre'] = 0
    
    return df_annot, Rheels_df, extension_df, bundles_df

In [17]:
"""remove data with low confidence"""
def removeLowConf(bundles_df):
    cofCols = group_headers(bundles_df, 'Cof_', True)
    confIndex = [False] * len(bundles_df.index)
    for col in cofCols:
        confIndex = confIndex | (bundles_df.loc[:,col] == 0)
    return bundles_df[~confIndex]

In [18]:
"""Plotting"""

'Plotting'

In [19]:
def detPlotColor(bundles_df, ind, colName):
    t = bundles_df.iloc[ind][colName]
    if t == 'T4':
        return 3
    elif t == 'T3':
        return 2
    elif t == 'T3_':
        return 2
    else:
        return -1

In [20]:
def getColorMap(cmapName, nGroup):
    cmap = plt.get_cmap(cmapName)
    colors = []
    gradient = np.linspace(0, 1, nGroup)
    for g in gradient:
        rgba = cmap(g)
        colors.append(matplotlib.colors.to_hex(rgba, keep_alpha = 'True'))
    return colors

In [21]:
def getLine(point, vector, span):
    return np.transpose(np.tile(span, (2,1))) * vector + point

In [22]:
def sumzip(*items):
    return [sum(values) for values in zip(*items)]

In [23]:
### plot one bundle:
def plotOneBundle(region_name, Rheels_df, df_annot, extension_df, bundles_df, no, isBundleNo):
    df_annot_sub = df_annot.loc[df_annot['Region_Name'] == region_name]

    RcellCoorColls =  group_headers(Rheels_df, 'coor_', True)
    
    if(isBundleNo):
        bundleNo = no
        ind = list(Rheels_df.index).index(bundleNo)
    else:
        ind = no
        bundleNo = Rheels_df.index[no]
    print(ind, bundleNo)
    
    points = np.reshape(Rheels_df.loc[bundleNo,RcellCoorColls].values, (6,2))
    center = np.array([Rheels_df.loc[bundleNo, 'shape_CenterX'],Rheels_df.loc[bundleNo, 'shape_CenterY']])
    axis = np.array([Rheels_df.loc[bundleNo, 'shape_MajorAxis'],Rheels_df.loc[bundleNo, 'shape_MinorAxis']])
    phi = Rheels_df.loc[bundleNo, 'shape_phi']
    ellipseXY =  getEllipseXY(center, axis, phi)

    # R3 and R4s
    cR3 = np.array([ Rheels_df.loc[bundleNo, 'coor_R3_X'], Rheels_df.loc[bundleNo, 'coor_R3_Y'] ])
    cR4 = np.array([ Rheels_df.loc[bundleNo, 'coor_R4_X'], Rheels_df.loc[bundleNo, 'coor_R4_Y'] ])

    ### plot points
    fix, ax = plt.subplots(1,1)
    for i in range(6):
        #ax.plot(points[i,0], points[i,1], marker = 'o', fillstyle='none', markeredgecolor = RcellColors[i], markeredgewidth = 0.2)
        ax.plot(points[i,0], points[i,1], color = RcellColors[i], marker = 'o', alpha = 0.5, markersize=12)

    ### plot ellipse
    ax.plot(center[0], center[1], color = '0.5', marker = 'o', markersize=12)
    ax.plot(ellipseXY[:,0], ellipseXY[:,1], color = '0.5')

    ### plot bundle axis
    bundle_axis = np.array([ Rheels_df.loc[bundleNo, 'bundle_axis_X'], Rheels_df.loc[bundleNo, 'bundle_axis_Y'] ])
    span = np.arange(0, 5, 1)
    cRm = (cR3 + cR4)/2
    line =  getLine(cRm, bundle_axis, span)
    ax.plot(line[:,0], line[:,1], color = '0.5', linewidth = 2.0)

    ### plot extensions
    isR3 = (not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R3pre'].values[0]))
    isR4 = (not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R4pre'].values[0]))
    if(isR3 | isR4):
        # putative target index
        iT3 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T3'].values[0].astype(int)
        iT4 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T4'].values[0].astype(int)
        iT7 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T7'].values[0].astype(int)
        # target position
        cT3 = np.array([Rheels_df.loc[iT3,'shape_CenterX'],Rheels_df.loc[iT3,'shape_CenterY']])
        cT4 = np.array([Rheels_df.loc[iT4,'shape_CenterX'],Rheels_df.loc[iT4,'shape_CenterY']])
        cT7 = np.array([Rheels_df.loc[iT7,'shape_CenterX'],Rheels_df.loc[iT7,'shape_CenterY']])
        # plot target
        ax.plot(cT3[0], cT3[1], marker = 'd', color = RcellColors[2], markersize=12)
        ax.plot(cT4[0], cT4[1], marker = 'd', color = RcellColors[3], markersize=12)
        ax.plot(cT7[0], cT7[1], marker = 'd', color = RcellColors[2], markersize=12)

        # R3-pre extension
        if(isR3):
            iE3 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo, 'Extension_R3pre'].values[0].astype(int)
            i3color = detPlotColor(bundles_df, ind, 'Extension_target_R3pre')
            scE3 = np.array([ extension_df.loc[iE3, 'Start_X'], extension_df.loc[iE3, 'Start_Y'] ])
            tcE3 = np.array([ extension_df.loc[iE3, 'End_X'], extension_df.loc[iE3, 'End_Y'] ])
            ax.plot(scE3[0], scE3[1], marker = '*', color = RcellColors[i3color], markersize=10)
            ax.plot(np.vstack((scE3,tcE3))[:,0], np.vstack((scE3,tcE3))[:,1], color = RcellColors[i3color], linewidth=2.0)

        # R4-pre extension
        if(isR4):
            iE4 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo, 'Extension_R4pre'].values[0].astype(int)
            i4color = detPlotColor(bundles_df, ind, 'Extension_target_R4pre')
            scE4 = np.array([ extension_df.loc[iE4, 'Start_X'], extension_df.loc[iE4, 'Start_Y'] ])
            tcE4 = np.array([ extension_df.loc[iE4, 'End_X'], extension_df.loc[iE4, 'End_Y'] ])
            ax.plot(scE4[0], scE4[1], marker = '*', color = RcellColors[i4color], markersize=10)
            ax.plot(np.vstack((scE4,tcE4))[:,0], np.vstack((scE4,tcE4))[:,1], color = RcellColors[i4color], linewidth=2.0)

    ax.set_title('Bundle No. ' + str(bundleNo))

In [24]:
### plot all
def plotAllBundlesInSubplots(region_name, bundles_df, Rheels_df, df_annot, extension_df, fSize, spacing):
    df_annot_sub = df_annot.loc[df_annot['Region_Name'] == region_name]
    
    RcellCoorColls =  group_headers(Rheels_df, 'coor_', True)
    
    # params for subplots
    numOfPlots = bundles_df.shape[0]
    n = round(np.sqrt(numOfPlots)).astype(int)
    
    fix, axes = plt.subplots(n, n, figsize=fSize)
    
    for ind, ax in zip(range(len(bundles_df.index)), axes.ravel()):
        print(ind)
        bundleNo = bundles_df.index[ind]

        points = np.reshape(Rheels_df.loc[bundleNo,RcellCoorColls].values, (6,2))
        center = np.array([Rheels_df.loc[bundleNo, 'shape_CenterX'],Rheels_df.loc[bundleNo, 'shape_CenterY']])
        axis = np.array([Rheels_df.loc[bundleNo, 'shape_MajorAxis'],Rheels_df.loc[bundleNo, 'shape_MinorAxis']])
        phi = Rheels_df.loc[bundleNo, 'shape_phi']
        ellipseXY =  getEllipseXY(center, axis, phi)

        ### plot points
        for i in range(6):
            #ax.plot(points[i,0], points[i,1], marker = 'o', fillstyle='none', markeredgecolor = RcellColors[i], markeredgewidth = 0.2)
            ax.plot(points[i,0], points[i,1], color = RcellColors[i], marker = 'o', alpha = 0.5, markersize=12)

        ### plot ellipse
        ax.plot(center[0], center[1], color = '0.5', marker = 'o', markersize=12)
        ax.plot(ellipseXY[:,0], ellipseXY[:,1], color = '0.5')

        ### plot bundle axis
        bundle_axis = np.array([ Rheels_df.loc[bundleNo, 'bundle_axis_X'], Rheels_df.loc[bundleNo, 'bundle_axis_Y'] ])
        span = np.arange(0, 5, 1)
        line =  getLine(center, bundle_axis, span)
        ax.plot(line[:,0], line[:,1], color = '0.5', linewidth = 2.0)

        ### plot extensions
        if((not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R3pre'].values[0])) | (not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R4pre'].values[0])) ):
            # putative target index
            iT3 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T3'].values[0].astype(int)
            iT4 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T4'].values[0].astype(int)
            iT7 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo,'Target_T7'].values[0].astype(int)
            # target position
            cT3 = np.array([Rheels_df.loc[iT3,'shape_CenterX'],Rheels_df.loc[iT3,'shape_CenterY']])
            cT4 = np.array([Rheels_df.loc[iT4,'shape_CenterX'],Rheels_df.loc[iT4,'shape_CenterY']])
            cT7 = np.array([Rheels_df.loc[iT7,'shape_CenterX'],Rheels_df.loc[iT7,'shape_CenterY']])
            # plot target
            ax.plot(cT3[0], cT3[1], marker = 'd', color = RcellColors[2], markersize=12)
            ax.plot(cT4[0], cT4[1], marker = 'd', color = RcellColors[3], markersize=12)
            ax.plot(cT7[0], cT7[1], marker = 'd', color = RcellColors[2], markersize=12)

            # R3-pre extension
            if(not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R3pre'].values[0])):
                iE3 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo, 'Extension_R3pre'].values[0].astype(int)
                i3color = detPlotColor(bundles_df, ind, 'Extension_target_R3pre')
                scE3 = np.array([ extension_df.loc[iE3, 'Start_X'], extension_df.loc[iE3, 'Start_Y'] ])
                tcE3 = np.array([ extension_df.loc[iE3, 'End_X'], extension_df.loc[iE3, 'End_Y'] ])
                ax.plot(scE3[0], scE3[1], marker = '*', color = RcellColors[i3color], markersize=10)
                ax.plot(np.vstack((scE3,tcE3))[:,0], np.vstack((scE3,tcE3))[:,1], color = RcellColors[i3color], linewidth=2.0)

            # R4-pre extension
            if(not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R4pre'].values[0])):
                iE4 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo, 'Extension_R4pre'].values[0].astype(int)
                i4color = detPlotColor(bundles_df, ind, 'Extension_target_R3pre')
                scE4 = np.array([ extension_df.loc[iE4, 'Start_X'], extension_df.loc[iE4, 'Start_Y'] ])
                tcE4 = np.array([ extension_df.loc[iE4, 'End_X'], extension_df.loc[iE4, 'End_Y'] ])
                ax.plot(scE4[0], scE4[1], marker = '*', color = RcellColors[i4color], markersize=10)
                ax.plot(np.vstack((scE4,tcE4))[:,0], np.vstack((scE4,tcE4))[:,1], color = RcellColors[i4color], linewidth=2.0)
        
        ### title of subplots
        ax.set_title(str(ind) + ', ' + str(bundleNo))
        
        
    plt.subplots_adjust(wspace = spacing[0], hspace=spacing[1])

In [25]:
### plot all
def plotAllBundlesTogether(region_name, Rheels_df, df_annot, extension_df, bundles_df, fSize):
    df_annot_sub = df_annot.loc[df_annot['Region_Name'] == region_name]
    
    RcellCoorColls = group_headers(Rheels_df, 'coor_', True)
    
    # params for subplots
    numOfPlots = Rheels_df.shape[0]
    n = round(np.sqrt(numOfPlots)).astype(int)
    
    fix, ax = plt.subplots(1,1, figsize=fSize)
    
    for ind in range(len(Rheels_df.index)):
        
        bundleNo = Rheels_df.index[ind]
        points = np.reshape(Rheels_df.loc[bundleNo,RcellCoorColls].values, (6,2))
        center = np.array([Rheels_df.loc[bundleNo, 'shape_CenterX'],Rheels_df.loc[bundleNo, 'shape_CenterY']])
        axis = np.array([Rheels_df.loc[bundleNo, 'shape_MajorAxis'],Rheels_df.loc[bundleNo, 'shape_MinorAxis']])
        phi = Rheels_df.loc[bundleNo, 'shape_phi']
        ellipseXY = getEllipseXY(center, axis, phi)

        ### plot points
        for i in range(6):
            #ax.plot(points[i,0], points[i,1], marker = 'o', fillstyle='none', markeredgecolor = RcellColors[i], markeredgewidth = 0.2)
            ax.plot(points[i,0], points[i,1], color = RcellColors[i], marker = 'o', alpha = 0.5, markersize=16)
        
        ### plot ellipse
        ax.plot(center[0], center[1], color = '0.5', marker = 'o', markersize=16)
        ax.plot(ellipseXY[:,0], ellipseXY[:,1], color = '0.5')
        
        ### plot bundle axis
        bundle_axis = np.array([ Rheels_df.loc[bundleNo, 'bundle_axis_X'], Rheels_df.loc[bundleNo, 'bundle_axis_Y'] ])
        span = np.arange(0, 5, 1)
        line = getLine(center, bundle_axis, span)
        ax.plot(line[:,0], line[:,1], color = '0.5', linewidth = 2.0)
        
        ### plot extensions
        if((not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R3pre'].values[0])) | (not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R4pre'].values[0])) ):

            # putative target index
            iT3 = df_annot_sub.iloc[ind]['Target_T3'].astype(int)
            iT4 = df_annot_sub.iloc[ind]['Target_T4'].astype(int)
            iT7 = df_annot_sub.iloc[ind]['Target_T7'].astype(int)
            # target position
            cT3 = np.array([Rheels_df.loc[iT3,'shape_CenterX'],Rheels_df.loc[iT3,'shape_CenterY']])
            cT4 = np.array([Rheels_df.loc[iT4,'shape_CenterX'],Rheels_df.loc[iT4,'shape_CenterY']])
            cT7 = np.array([Rheels_df.loc[iT7,'shape_CenterX'],Rheels_df.loc[iT7,'shape_CenterY']])
            # plot target
            #ax.plot(cT3[0], cT3[1], marker = 'd', color = RcellColors[2], markersize=12)
            #ax.plot(cT4[0], cT4[1], marker = 'd', color = RcellColors[3], markersize=12)
            #ax.plot(cT7[0], cT7[1], marker = 'd', color = RcellColors[2], markersize=12)

            # R3pre extension
            # R3-pre extension
            if(not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R3pre'].values[0])):
                iE3 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo, 'Extension_R3pre'].values[0].astype(int)
                i3color = detPlotColor(bundles_df, ind, 'Extension_target_R3pre')
                scE3 = np.array([ extension_df.loc[iE3, 'Start_X'], extension_df.loc[iE3, 'Start_Y'] ])
                tcE3 = np.array([ extension_df.loc[iE3, 'End_X'], extension_df.loc[iE3, 'End_Y'] ])
                ax.plot(scE3[0], scE3[1], marker = '*', color = RcellColors[i3color], markersize=10)
                ax.plot(np.vstack((scE3,tcE3))[:,0], np.vstack((scE3,tcE3))[:,1], color = RcellColors[i3color], linewidth=2.0)

            # R4-pre extension
            if(not pd.isnull(df_annot_sub.loc[list(df_annot_sub['Bundle_No'] == bundleNo),'Extension_R4pre'].values[0])):
                iE4 = df_annot_sub.loc[df_annot_sub['Bundle_No'] == bundleNo, 'Extension_R4pre'].values[0].astype(int)
                i4color = detPlotColor(bundles_df, ind, 'Extension_target_R3pre')
                scE4 = np.array([ extension_df.loc[iE4, 'Start_X'], extension_df.loc[iE4, 'Start_Y'] ])
                tcE4 = np.array([ extension_df.loc[iE4, 'End_X'], extension_df.loc[iE4, 'End_Y'] ])
                ax.plot(scE4[0], scE4[1], marker = '*', color = RcellColors[i4color], markersize=10)
                ax.plot(np.vstack((scE4,tcE4))[:,0], np.vstack((scE4,tcE4))[:,1], color = RcellColors[i4color], linewidth=2.0)

        ax.set_title('Overview')

In [26]:
def plotScatter(X, y, classes, colors, title, xlabel, ylabel):
    fig = plt.figure()
    ax = plt.subplot(111)

    for color, i, target_name in zip(colors, classes, classes):
        plt.scatter(X[y == i, 0], X[y == i, 1], color=color,
                    label=target_name)
    plt.legend(loc='best', shadow=False, scatterpoints=1)

    # Shrink current axis by 20%
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

    # Put a legend to the right of the current axis
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()